In [ ]:
from PIL import Image
import torch
import torchvision
import torchvision.models as models

import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
from torch.nn import functional as F
import os

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Dropout(0.5),
               nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/binary/models/weightsV3.h5'), strict=False)

In [ ]:
def predict_multiple_images(folder_path):
    target_image_paths = []  # List to store paths of "Target" images

    # List all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        try:
            # Open and process the image
            img = Image.open(file_path).convert('RGB')
            img_tensor = data_transform(img).unsqueeze(0)
            model.eval()

            # Predict the image
            with torch.no_grad():
                pred_logits_tensor = model(img_tensor.to(device))
                pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()

                # Print prediction probabilities for the current image
                print(f"Prediction probabilities for {filename}: {pred_probs}")

                ans = np.argmax(pred_probs, axis=1)

                # Display result
                if ans == 0:
                    print(f"{filename}: Non-Target")
                else:
                    print(f"{filename}: Target")
                    target_image_paths.append(file_path)  # Collect target image path

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    # Return or use target image paths for further predictions (e.g., object detection)
    return target_image_paths

In [ ]:
# Example usage
folder_path = "/content/drive/MyDrive/new non-target"
predict_multiple_images(folder_path)

In [ ]:
model = YOLO('best.pt')
names = model.names

In [ ]:
print("Collected Target Image Paths:", target_paths)

In [ ]:
results = model.predict(im,conf=0.5,iou=0.5)
class_data = []  # Create a list of all Classes

for box, cls, conf in zip(results[0].boxes.xyxy, results[0].boxes.cls, results[0].boxes.conf):
    x1, y1, x2, y2 = map(int, box.tolist())
    class_id = int(cls)
    class_name = names[class_id]
    confidence = float(conf)
    class_data.append({
        'class': class_name,
        'conf': confidence,
        'position': (x1, y1, x2, y2)
    })

# Check if no objects were detected
if not class_data:
    message = "=============================================================================================\n"
    message += f"{filename}: ไม่สามารถจำแนกได้"
else:
    # Sort the detections by confidence score in descending order
    class_data.sort(key=lambda x: x['conf'], reverse=True)

    # Define RGB colors for each class BGR
    class_colors = {
        'Healthy': (0, 140, 255),  # orange
        'LFNS': (0, 0, 255),  # red
        'LFS': (204, 0, 51),  # blue
        'PM': (255, 0, 204),  # purple
        'CoryL': (0, 204, 204),  # yellow
        'BES': (147, 20, 255),  # pink
        'CLS': (255, 255, 0),  # light blue
    }

    # Create a dictionary to aggregate class information
    class_info = defaultdict(lambda: {'max_conf': 0, 'count': 0, 'detections': []})

    # Flag to check if any non-Healthy class is detected
    non_healthy_detected = False

    # Iterate over the class data and update the class information
    for data in class_data:
        class_name = data['class']
        confidence = data['conf']

        if class_name != 'Healthy':
            non_healthy_detected = True

        # Append detections per class for later sorting
        class_info[class_name]['detections'].append(data)

    # Now sort the detections within each class by confidence and limit to top 3
    for class_name, info in class_info.items():
        info['detections'] = sorted(info['detections'], key=lambda x: x['conf'], reverse=True)[:3]

    # Draw the bounding boxes for the top 3 detections per class
    for class_name, info in class_info.items():
        # Skip drawing the bounding box for the 'Healthy' class if non_healthy_detected is True
        if class_name == 'Healthy' and non_healthy_detected:
            continue

        for data in info['detections']:
            # Extract bounding box coordinates and color
            x1, y1, x2, y2 = data['position']
            confidence = data['conf']
            color = class_colors.get(class_name, (128, 128, 128))

            # Draw the bounding box
            cv2.rectangle(im0, (x1, y1), (x2, y2), color, 2)

            # Prepare the label with the class name and confidence
            label = f'{class_name} {confidence:.2f}'
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.7
            font_thickness = 2

            # Get the label size
            (label_width, label_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)

            # Draw a filled rectangle for the label background
            cv2.rectangle(im0, (x1, y1 - label_height - 10), (x1 + label_width, y1), color, -1)

            # Draw the label text
            cv2.putText(im0, label, (x1, y1 - 5), font, font_scale, (255, 255, 255), font_thickness)

    # Prepare a message for the results (you can keep your existing message formatting logic here)
    sorted_class_info = sorted(class_info.items(), key=lambda x: x[1]['detections'][0]['conf'], reverse=True)
    message = "=============================================================================================\n"
    message += f"{filename}: ผลการจำแนก\n"
    for class_name, info in sorted_class_info:
        if class_name == 'Healthy' and non_healthy_detected:
            continue  # Skip Healthy if other classes are detected
        elif class_name == 'Healthy' and not non_healthy_detected:
            message += f"- สุขภาพดี ({class_name}) {info['detections'][0]['conf']*100:.0f}%\n"
        else:
            if class_name == 'LFNS':
                message += f"- โรคใบร่วงชนิดใหม่แผลดอกไม้ ({class_name}) {info['detections'][0]['conf']*100:.0f}%\n"
            elif class_name == 'LFS':
                message += f"- โรคใบร่วงชนิดใหม่แผลจุดกลม ({class_name}) {info['detections'][0]['conf']*100:.0f}%\n"
            elif class_name == 'PM':
                message += f"- โรคราแป้ง ({class_name}) {info['detections'][0]['conf']*100:.0f}%\n"
            elif class_name == 'CoryL':
                message += f"- โรคใบจุดก้างปลา ({class_name}) {info['detections'][0]['conf']*100:.0f}%\n"
            elif class_name == 'BES':
                message += f"- โรคใบจุดตานก ({class_name}) {info['detections'][0]['conf']*100:.0f}%\n"
            elif class_name == 'CLS':
                message += f"- โรคใบจุดนูน ({class_name}) {info['detections'][0]['conf']*100:.0f}%\n"

print(message)